## ***PARAMETROS DE ENTRADA***

In [88]:
#Diretórios e parâmetros de filtragem

folder_path = '/content/drive/MyDrive/GE_AOI'         #endereço da pasta do Google Drive onde estão os shapefiles
result_folder_name = 'RESULTADOS'                     #nome da pasta de resultados
data_inicio_periodo = '2022-02-01'                    #data de inicio do periodo de análise
data_fim_periodo = '2023-02-01'                       #data de inicio do periodo de análise
porc_nuvens = 30.0                                    #porcentagem de nuvens

nome_projeto_gee = 'ee-mapeamentofrigg'               #nome do projeto do GEE para autenticação

## ***INSTALAÇÃO E IMPORTAÇÃO DE BIBLIOTECAS***

In [ ]:
#Instala bilbiotecas necessárias
!pip install earthengine-api
!pip install geemap
!pip install PyCRS==0.1.3
!pip install filetype
!pip install xlsxwriter
!pip install kora

In [119]:
#Importa bibliotecas necessárias
import os
import ee
import geemap as geemap

import geopandas as gpd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipyleaflet import WidgetControl,Map, LegendControl

import shapely
from shapely.validation import make_valid, explain_validity
from shapely.geometry import Polygon
from shapely.ops import cascaded_union

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth,drive,files
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
from kora.xattr import get_id

from datetime import datetime,timedelta, date

import xlsxwriter

## ***AUTENTICAÇÃO ECONEXÃO DO GOOGLE DRIVE / COLAB / EARTH ENGINE***

In [70]:
# Autentica e inciliza o GEE
ee.Authenticate()
ee.Initialize(project=nome_projeto_gee)

# Autentica Conecta o Colab ao Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Mounted at /content/drive


## ***LIMPEZA DA PASTA DE RESULTADOS***

In [73]:
# Funções para verificar se a pasta "RESULTADOS" existe no Google Drive. Caso não exista, cria-a.

# Define o nome da pasta a ser checkada
folder_name = result_folder_name

# Função para verificar se existe uma pasta e retornar seu id
def get_folder_id(folder_name):
    query = f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
    file_list = drive.ListFile({'q': query}).GetList()
    if file_list:
        return file_list[0]['id']
    else:
        return None

#Função para deletar uma pasta pelo seu id
def delete_folder(folder_id):
    folder = drive.CreateFile({'id': folder_id})
    folder.Delete()

#Função para criar uma nova pasta
def create_folder(folder_name):
    folder_metadata = {
        'title': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()
    print(f'Pasta "{folder_name}" criada com o ID: {folder["id"]}')

# Verifica se a pasta existe
folder_id = get_folder_id(folder_name)

if folder_id:
    print(f'Pasta "{folder_name}" já existe com o ID: {folder_id}')
    print('Apagando a pasta...')
    delete_folder(folder_id)
    print(f'A pasta "{folder_name}" foi deletada.')
else:
    print(f'A pasta "{folder_name}" não existe.')

# Cria um nova pasta
create_folder(folder_name)

Pasta "RESULTADOS" já existe com o ID: 1ZqnhcmB-nC26FwXBZFIJiQhENRTTPq5c
Apagando a pasta...
A pasta "RESULTADOS" foi deletada.
Pasta "RESULTADOS" criada com o ID: 1rF_m20JeKYP7Ygx9G8TTefF7XZxVnc6Z


## ***FUNÇÕES DE PROCESSAMENTO***

In [160]:
#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para calcular a másca de núvens
def maskCloudAndShadowsSR(image):
    cloudProb = image.select('MSK_CLDPRB')
    snowProb = image.select('MSK_SNWPRB')
    cloud = cloudProb.lt(5)
    snow = snowProb.lt(5)
    scl = image.select('SCL')
    shadow = scl.eq(3)
    cirrus = scl.eq(10)
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
    return image.updateMask(mask)
#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para calcular NDVI de uma imagem
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)
#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para calcular o valor médio e o desvio padrão do NDVI dentro da AOI, e associar cada valor à data da imagem
def datedist(image):
 img = ee.Image(image)
 date = img.get('system:time_start')
 NDVI_band = img.select('NDVI')
 NDVI_mean = NDVI_band.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi, scale=10)
 NDVI_mean_value = NDVI_mean.get('NDVI')
 NDVI_stdv = NDVI_band.reduceRegion(reducer=ee.Reducer.stdDev(), geometry=aoi, scale=10)
 NDVI_stdv_value = NDVI_stdv.get('NDVI')
 return img.set('DateDist', date).set('NDVI_mean', NDVI_mean_value).set('NDVI_stdv', NDVI_stdv_value)
#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para classificar o NDVI de uma imagem em "ALTO/MEDIO/BAIXO"
def classify_ndvi(image):
    ndvi_class= image.select("NDVI").where(image.select("NDVI").lt(0.0), 0).where(image.select("NDVI").gte(0.0).And(image.select("NDVI").lt(0.3)), 1).where(image.select("NDVI").gte(0.3).And(image.select("NDVI").lt(0.6)), 2).where(image.select("NDVI").gte(0.6), 3).rename('NDVI_CLASS').clip(aoi)
    return image.updateMask(ndvi_class).addBands(ndvi_class)

#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para exportar uma planilha de Excel contendo os valores de NDVI da série temporal e um grafico exploratório
def export_xls(button):
  #Cria um arquivo excel no Google Drive.
  writer = pd.ExcelWriter((result_path + '/resultados.xlsx'), engine="xlsxwriter")

  # Converte o dataframe para um objeto XlsxWriter.
  df.to_excel(writer, sheet_name="Sheet1")

  #Define o workbook e a worksheet
  workbook = writer.book
  worksheet = writer.sheets["Sheet1"]

  # Cria um objeto de gráfico.
  chart = workbook.add_chart({"type": "line"})
  (max_row, max_col) = df.shape

  # Confiurações do gráfico.
  chart.add_series({"name": "NDVI_mean",'categories':'=Sheet1!B2:B10000', 'values': '=Sheet1!C2:C10000',"y2_axis": 1})
  chart.add_series({"name": "NDVI_stdv",'categories':'=Sheet1!B2:B10000', 'values': '=Sheet1!D2:D10000'})
  chart.set_legend({'position': 'right'})
  chart.set_title({"name": "Variação de NDVI"})
  chart.set_x_axis({"name": "Data",'date_axis': True})
  chart.set_y_axis({"name": "NDVI_mean", "major_gridlines": {"visible": 0}})
  chart.set_y2_axis({"name": "NDVI_stdv"})

  # Insere o gráfico na worksheet
  worksheet.insert_chart('F2', chart)
  writer.close()
  print("Exporting Arquivo XLS...")


#________________________________________________________________________________________________________________________________________________________________________________________#
# Função adicionar um imagem de NDVI e RGB ao Mapa na data selecionada

def export_image(button):
    selected_start_date = str(date_dropdown.value).split(' ')[0]
    selected_end_date = (str(date_dropdown.value + timedelta(days=1)).split(' ')[0])

    #filtra a imagem em função da data escolhida
    selected_img = s2_imgcol.filterDate(selected_start_date, selected_end_date)

    #Calcula a médiana dos valores na data escolhida
    ndvi_image = selected_img.select(['NDVI']).median()

    #Exporta a imagem de NDVI para o Google Drive
    export_image_task = ee.batch.Export.image.toDrive(
        image=ndvi_image.clip(aoi),
        description='NDVI_IMG_' + selected_start_date.replace("-",""),
        folder= result_folder_name,
        scale=10,
        region=aoi.geometry(),
        fileFormat='GeoTIFF')

    #Inicia exportação
    export_image_task.start()

    print(f"Exportando imagem de NDVI de {selected_start_date}...")

#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para remover todos os layers

def remove_layers(button):
    layers_to_keep = ['AOI']
    layers = list(Map.layers)
    for layer in layers:
        if layer.name not in layers_to_keep:
            Map.remove_layer(layer)


#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para exportar uma imagem de NDVI para a data selecionada
def add_image(button):
    selected_start_date = str(date_dropdown.value).split(' ')[0]
    selected_end_date = (str(date_dropdown.value + timedelta(days=1)).split(' ')[0])

    #filtra a imagem em função da data escolhida
    selected_img = s2_imgcol.filterDate(selected_start_date, selected_end_date).median()

    #Calcula a médiana dos valores na data escolhida
    Map.add_basemap('OpenStreetMap')
    Map.addLayer(selected_img.clip(aoi),{'min':0, 'max':0.9,'bands':'NDVI','palette': ['red','yellow','green']},"NDVI_"+ selected_end_date)
    Map.addLayer(selected_img.clip(aoi),{'min':1, 'max':3,'bands':'NDVI_CLASS','palette': ['red','yellow','green']},"NDVI_CLASS"+ selected_end_date)
    Map.addLayer(selected_img,{'min':150, 'max':4500,'bands':['B11','B8','B3']},"RGB_"+ selected_end_date)


#________________________________________________________________________________________________________________________________________________________________________________________#
# Função para exportar  os poligonos da classificação do NDVI para a data selecionada
def export_vector(button):
    selected_start_date = str(date_dropdown.value).split(' ')[0]
    selected_end_date = (str(date_dropdown.value + timedelta(days=1)).split(' ')[0])

    #filtra a imagem em função da data escolhida
    selected_img = s2_imgcol.filterDate(selected_start_date, selected_end_date)

    #Calcula a médiana dos valores na data escolhida
    ndvi_class_image = selected_img.select(['NDVI_CLASS']).median()

    #Converte a imagem de NDVI em poligonos
    ndvi_vector = ndvi_class_image.toInt().reduceToVectors(
        geometry = aoi.geometry(),
        geometryType='polygon',
        labelProperty= 'NDVI_CLASS',
        scale=10,
        maxPixels=1e8)

    #Exporta os poligonos de classes de NDVI para o Google Drive
    export_vector_task = ee.batch.Export.table.toDrive(
        collection=ndvi_vector,
        description='NDVI_POL_' + selected_start_date.replace("-",""),
        folder= result_folder_name,
        fileFormat='SHP')

    #Inicia exportação
    export_vector_task.start()

    print(f"Exportando vetores de NDVI de {selected_start_date}...")

#________________________________________________________________________________________________________________________________________________________________________________________#

## ***PROCESSAMENTO***

In [161]:
#Caminho para a pasta de resultados
result_path = '/content/drive/MyDrive/' + result_folder_name

#Verifica os shapefilaes existentes na pasta de shapefiles
path=[]
def list_shps_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
          if file.split(".")[-1] == "shp":
            path.append(os.path.join(root,file))
list_shps_in_folder(folder_path)

# Converte os poligonos para geodataframe e aplica "merge" para compor a área de interesse (AOI)
gdf_imoveis = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path],ignore_index=True), crs=gpd.read_file(path[0]).crs)

# Corrige geometrias inválidas
shapely.make_valid(gdf_imoveis.geometry)

# Converte de GeoDataframe para ee.FeatureCollection
aoi = geemap.gdf_to_ee(gdf_imoveis)

#Carrega as imagens Sentinel2- Harmonizada, aplica filtros de seleção e aplica funções acima
s2_imgcol = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
              .filterBounds(aoi)
              .filterDate(data_inicio_periodo,data_fim_periodo)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', porc_nuvens))
              .map(maskCloudAndShadowsSR)
              .map(calculate_ndvi)
              .map(datedist)
              .map(classify_ndvi))

# Cria um map para visualizar os dados
Map = geemap.Map()

# Aproxima a visualização à AOI
Map.centerObject(aoi, zoom= 15)

# Cria array de datas e valores para compor o dataframe da série temporal
dates_array = s2_imgcol.aggregate_array('DateDist')
NDVI_mean_array = s2_imgcol.aggregate_array('NDVI_mean')
NDVI_stdv_array = s2_imgcol.aggregate_array('NDVI_stdv')

# Cria listas de datas e valores para compor o dataframe da série temporal
dates_list = dates_array.getInfo()
NDVI_mean_list = NDVI_mean_array.getInfo()
NDVI_stdv_list = NDVI_stdv_array.getInfo()
string_timestamps = [datetime.utcfromtimestamp(int(date_str) // 1000) for date_str in dates_list]

# Adiciona as imagens de NDVI no mapa com um time slider
start_date = str(string_timestamps[-1]).split(' ')[0]
end_date = (str(string_timestamps[-1] + timedelta(days=1)).split(' ')[0])
end_img = s2_imgcol.filterDate(start_date, end_date).median()
Map.addLayer(end_img,{'min':0, 'max':0.9,'bands':'NDVI','palette': ['red','yellow','green']},"NDVI_"+ end_date)
Map.addLayer(end_img,{'min':1, 'max':3,'bands':'NDVI_CLASS','palette': ['red','yellow','green']},"NDVI_CLASS"+ end_date)
Map.addLayer(end_img,{'min':150, 'max':4500,'bands':['B11','B8','B3']},"RGB_"+ end_date)

# Adiciona o perímetro da AOI no mapa
aoi_polygon = ee.Image().paint(aoi, 0, 2)
Map.addLayer(aoi_polygon, {'palette': 'black'}, 'Polígonos')

# Cria do dataframe da série temporal
data = {'Date': string_timestamps, 'NDVI_mean': NDVI_mean_list,'NDVI_stdv': NDVI_stdv_list}
df = pd.DataFrame(data)

#Configurações do gráfico
plt.figure(figsize=(5, 3))
mean_line, = plt.plot(df['Date'], df['NDVI_mean'], marker='.',markersize=8, linestyle='-', color='r')
plt.title('Variação do NDVI')
plt.ylabel('NDVI_mean')
plt.ylim((0,1.0))
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
ax2 = plt.twinx()
stdv_line, = ax2.plot(df['Date'], df['NDVI_stdv'], marker='^',markersize=5, linestyle='-', color='b')
ax2.set_ylabel('NDVI_stdv')
plt.legend([mean_line, stdv_line], ['NDVI_mean', 'NDVI_stdv'], loc='upper right')

# Cria um widget para armazenar o gráfico e plota o gráfico
output_widget = widgets.Output()
with output_widget:
    output_widget.clear_output()
    plt.show()

# Cria um widget para armazenar a legenda do NDVI
legend = LegendControl({"Baixo":"red", "Médio":"yellow", "Alto":"green"}, title="NDVI", position="bottomleft")
Map.add(legend)

# Cria um widget de lista dropdown com as datas das imagens disponíveis
date_dropdown = widgets.Dropdown(
    options=string_timestamps,
    description='Selecione uma Data:',
    disabled=False,)

#Cria um widget de botão para adicionar uma imagem de NDVI e RGB no mapa
add_img_button = widgets.Button(
    description='Adicionar Imagens',
    disabled=False,
    button_style='',
    tooltip='Adiciona Imagens ao Mapa',
    icon='plus-square',)

#Cria um widget de botão para remover uma imagem de NDVI e RGB no mapa
remove_img_button = widgets.Button(
    description='Remove Imagens',
    disabled=False,
    button_style='',
    tooltip='Remove imagens do Mapa',
    icon='minus-square',)

#Cria um widget de botão para exportar imagem NDVI para o Google Drive
export_tiff_button = widgets.Button(
    description='Export GEOTIFF',
    disabled=False,
    button_style='',
    tooltip='Click para exportar NDVI image to Google Drive ',
    icon='cloud-download',)

#Cria um widget de botão para exportar poligonos de NDVI para o Google Drive
export_pol_button = widgets.Button(
    description='Export Polygons',
    disabled=False,
    button_style='',
    tooltip='Click para exportar poligonos de NDVI para o Google Drive',
    icon='cloud-download',)

#Cria um widget de botão para exportar XLS
export_xls_button = widgets.Button(
    description='Exportar XLSX',
    disabled=False,
    button_style='',
    tooltip='Click para exportar XLS da Série Temporal para o Google Drive',
    icon='file-archive-o',)

#Atribui ação do botão
add_img_button.on_click(add_image)
export_tiff_button.on_click(export_image)
export_pol_button.on_click(export_vector)
export_xls_button.on_click(export_xls)
remove_img_button.on_click(remove_layers)

output_control = WidgetControl(widget=output_widget, position="bottomright")
Map.add_control(output_control)

Map.addLayerControl()


## ***EXIBIR RESULTADOS***

In [162]:
#Exibe mapa
display(widgets.HBox([date_dropdown,add_img_button,remove_img_button, export_tiff_button,export_pol_button, export_xls_button]),Map)

Map(center=[-21.45637886012685, -54.89226233662243], controls=(WidgetControl(options=['position', 'transparent…

OpenStreetMap has been already added before.
OpenStreetMap has been already added before.


OpenStreetMap has been already added before.
Exportando imagem de NDVI de 2022-07-07...
Exportando vetores de NDVI de 2022-07-07...
Exporting Arquivo XLS...
